<a href="https://colab.research.google.com/github/NaraaBold/DiverNaraa/blob/main/Recurrent_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Implementation of recurrent neural network (RNN) from scratch

[Problem 1] Forward propagation implementation of SimpleRNN

In [78]:
import numpy as np
x = np.array([[[1, 2], [2, 3], [3, 4]]])/100 # (batch_size, n_sequences, n_features)
w_x = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100 # (n_features, n_nodes)
w_h = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100 # (n_nodes, n_nodes)
batch_size = x.shape[0] # 1
n_sequences = x.shape[1] # 3
n_features = x.shape[2] # 2
n_nodes = w_x.shape[1] # 4
h = np.zeros((batch_size, n_nodes)) # (batch_size, n_nodes)
b = np.array([1, 1, 1, 1]) # (n_nodes,)

In [79]:
def forward(x,h):
    for n in range(n_sequences):
        h = np.tanh(x[:, n, :] @ w_x + h @ w_h + b)
    return h

In [80]:
forward(x,h)

array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]])

[Problem 1] Forward propagation implementation of SimpleRNN

In [8]:
#Defining SimpleRNN class
class SimpleRNN:
    def __init__(self, batch_size, n_features, n_sequences, n_nodes):
        self.batch_size = batch_size
        self.n_features = n_features
        self.n_nodes = n_nodes
        self.n_sequences = n_sequences
        self.w_x = None
        self.w_h = None
        self.b = None
    def forward(self, x, h):
        for n in range(self.n_sequences):
          a_t = x[:, n, :] @ self.w_x + h @ self.w_h + self.b
          #print (a_t.shape)
          h = np.tanh(a_t)
        return h


[Problem 2] Forward propagation experiment with small sequences

In [9]:
rnn = SimpleRNN(batch_size, n_features, n_sequences, n_nodes)
rnn.w_x = w_x
rnn.w_h = w_h
rnn.b = b
result=rnn.forward(x,h)
print (result)

[[0.79494228 0.81839002 0.83939649 0.85584174]]


[Problem 3] (Advanced assignment) Implementation of back propagation

In [91]:
#Defining SimpleRNN class
class SimpleRNN2:
    def __init__(self, batch_size, n_features, n_sequences, n_nodes, lr = 0.01):
        self.batch_size = batch_size
        self.n_features = n_features
        self.n_nodes = n_nodes
        self.n_sequences = n_sequences
        self.w_x = None
        self.w_h = None
        self.b = None
        self.a_t = np.zeros((n_sequences, batch_size, n_nodes)) # (n_sequences, batch_size, n_nodes)
        self.h_t = np.zeros((n_sequences, batch_size, n_nodes)) # (n_sequences, batch_size, n_nodes)
        self.dZ = np.zeros((n_sequences, batch_size, n_features))
        self.lr = lr
        
    def forward(self, x, h):
        self.X = x
        for n in range(self.n_sequences):
          a = x[:, n, :] @ self.w_x + h @ self.w_h + self.b
          self.a_t[n, :, : ] = a
          h = np.tanh(a)
          self.h_t[n,: , :] = h
        return h
    
    def backward(self, dA):
        for n in range(self.n_sequences-1, -1, -1):
          dA = dA * (1 - np.tanh(self.a_t[n, :, :])**2)
          dW_X = self.X[:, n, :].T @ dA
          dW_H = self.h_t[n,:,:].T @ dA
          Z = dA @ self.w_h.T
          self.dZ[n,:,:] = Z @ self.w_x.T
          dB = np.sum(dA, axis=0)
          #print (dZ.shape)
          #print (dW_X.shape)
          #print (self.w_x.shape)
          self.w_x -= self.lr * dW_X
          self.w_h -= self.lr * dW_H
          #print (dB)
          self.b -= self.lr * dB
        return self.dZ

    def tanh(self,X):
      result = (np.exp(X)-np.exp(-X))/(np.exp(X)+np.exp(-X))
      # or
      #  result = np.tanh(X)
      return result

In [92]:
rnn2 = SimpleRNN2(batch_size, n_features, n_sequences, n_nodes)
rnn2.w_x = w_x
rnn2.w_h = w_h
rnn2.b = b.astype(float)
result = rnn2.forward(x,h)
print (result)
rnn2.backward(result)
print (rnn2.dZ)

[[0.79036637 0.81462841 0.83631377 0.85324984]]
[[[0.0011638  0.00158902]]

 [[0.00282138 0.00385511]]

 [[0.00930855 0.01277143]]]
